## Notebook to explore packages for image processing and human face bounding.

In [5]:
import cv2
import glob
import numpy as np

Overall basic function calls for any cv2 project.
 - The color of the image can be changed with second param, greyscale/BW/color in the .imread() function.
 - The .imshow() function to get GUI to show image, first param is window title and second is image array. 
 - The .waitkey() function to hold window on screen, param for int is milliseconds, 0 is until user exits. 
 - The destroyAllWindows() function close all open cv2 windows.

In [6]:
# Loading and importing the image we want to analyze.
image_path = "C:/Users/rboulet.TOTALSYSTECH/OneDrive - Total Systems Technologies Corporation (TSTC)/Face Recognition/faces_in_wild_data/Bill_Gates/Bill_Gates_0001.jpg"
image_orig = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Convert the image we are analyzing using the haarcascades analyzer (pretrained).
image_grey = cv2.cvtColor(image_orig, cv2.COLOR_RGB2GRAY)

# Loading and applying the haarcascades analyzer from cv2 library.
face_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_alt.xml")
# Assigning face(s) found to python object to draw on original image.
detected_objects = face_classifier.detectMultiScale(image_grey, minSize=(50,50))

# Draw bounding box based on the face locations identified using the cascade classifier.
# Docs for detected_objects returned object = (x, y, width, height).
# Docs for drawing rectangle cv2 = cv2.rectangle(image, start_point:{x,y}, end_point, color, thickness)
for (x, y, width, height) in detected_objects:
    cv2.rectangle(image_orig, 
                  (x,y),
                  (x + height, y + width),
                  (0, 0, 250),
                  4)


# Show the original image with the rectangle for face drawn on it.
cv2.imshow('Bill', image_orig)
# Needed so kernel does not crash as well.
cv2.waitKey(0)
# This will just return -1 if not included vs kill window text, possibly for ensuring no background processes?
cv2.destroyAllWindows()

Putting the cell above into a function to call on a specific folder to see how it performs on all images.

In [10]:
# Using list comprehension to read all images (.jpg wildcard) to an object for Bill Gates.
bill_folder_all_jpg = 'faces_in_wild_data\lfw-deepfunneled\lfw-deepfunneled\Bill_Gates\*.jpg'
bill_images = [cv2.imread(file) for file in glob.glob(bill_folder_all_jpg)]

In [11]:
print(len(bill_images))

17


In [16]:
# Loading and applying the haarcascades analyzer from cv2 library.
face_classifier = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_alt.xml")

# Function to source different folders for facial bounding and return windows one after the other with results.
def bound_faces(person: str) -> object:
    person_folder_path = f'faces_in_wild_data\lfw-deepfunneled\lfw-deepfunneled\{person}\*.jpg'
    person_images = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob(person_folder_path)]

    for image in person_images:
        # Convert original image to greyscale.
        image_grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Assigning face position found from facial classifier to draw on original image.
        face_location = face_classifier.detectMultiScale(image_grey, minSize=(10,10))
        
        # Draw bounding box based on the face locations identified using the cascade classifier.
        for (x, y, width, height) in face_location:
            cv2.rectangle(image, 
                          (x,y),
                          (x + height, y + width),
                          (0, 250, 0),
                          2)
        
        # Show the original image with the rectangle for face drawn on it.
        cv2.imshow(person, image)
        # Needed so kernel does not crash as well.
        cv2.waitKey(0)
        # This will just return -1 if not included vs kill window text, possibly for ensuring no background processes?
        cv2.destroyAllWindows()

    return 'All Images Complete.'

In [18]:
bound_faces('colin_Powell')

Looking into face classification for testing whether or not images of persons are correctly identified based on input classes.
The quesstions I have currently on this process:
1. Is only a portion of the image used, e.g. the face when bounded is extracted, when classifying images of persons?
2. What is a reasonable number of classes to attempt?
3. Are there impacts to performance when an imbalanced training dataset is used.
4. Can this be performed on both images and video?

<u> Info pertaining to haarcascades classifier algorithm: </u>

_"This is done using the cv2::CascadeClassifier::detectMultiScale method, which returns boundary rectangles for the detected faces (i.e., x, y, w, h). It takes two parameters namely, scaleFactor and minNeighbors. ScaleFactor determines the factor of increase in window size which initially starts at size “minSize”, and after testing all windows of that size, the window is scaled up by the “scaleFactor”, and the window size goes up to “maxSize”. If the “scaleFactor” is large, (e.g., 2.0), there will be fewer steps, so detection will be faster, but we may miss objects whose size is between two tested scales. (default scale factor is 1.3). Higher the values of the “minNeighbors”, less will be the number of false positives, and less error will be in terms of false detection of faces. However, there is a chance of missing some unclear face traces as well."_

__1. Is only a portion of the image used, e.g. the face when bounded is extracted, when classifying images of persons?__

 - The SIFT method is one of the techniques used to extract features of a face in order to compare to others. SIFT is built into opencv library. 
 - SIFT is can be used regardless of size of image and orientation of image.
 - FAST is another method of feature extraction used for images. It is a corner detection method (LOOK UP) and is faster than SIFT which is useful when doing things like real time video capture and comparison.
 - ORB is another method to extract features. This is 'rotation invariant' (LOOK UP). SIFT and SURF are patented and this is OSS full license, and is apparently more efficient than SIFT and SURF (LOOK UP SURF).

__Exploring SIFT__

In [14]:
# Loading SIFT algorithm from opencv library.
sift = cv2.SIFT_create()

# Image path.
image_path = 'faces_in_wild_data\lfw-deepfunneled\lfw-deepfunneled\Bill_Gates\Bill_Gates_0001.jpg'

# Read original color image. Convert color image to gray and assign.
image_orig = cv2.imread(image_path, cv2.IMREAD_COLOR)
image_gray = cv2.cvtColor(image_orig, cv2.COLOR_RGB2GRAY)

# Applying the SIFT algorithm to the gray image.
keypoints = sift.detect(image_gray, None) # Check docs on method called and inputs/ouptuts.

# Draw identified keypoints on the gray image.(LOOK UP this relative to previous without size).
image_with_keypoints = cv2.drawKeypoints(image_gray, 
                                         keypoints, 
                                         image_orig, 
                                         flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Show the original image with the rectangle for face drawn on it.
cv2.imshow('keypoints on image', image_with_keypoints)
# Needed so kernel does not crash as well.
cv2.waitKey(0)
# This will just return -1 if not included vs kill window text, possibly for ensuring no background processes?
cv2.destroyAllWindows()

__Exploring FAST algorithm for feature extraction__

In [15]:
# Load FAST algorithm from opencv library.
fast = cv2.FastFeatureDetector_create()

# Detecting keypoints with FAST. 
keypoints_with_nonmax = fast.detect(image_gray, None)


image_fast_keypoints = cv2.drawKeypoints(image_gray, 
                                 keypoints_with_nonmax, 
                                 image_orig, 
                                 color=(0,255,0),
                                 flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


# Show the original image with the rectangle for face drawn on it.
cv2.imshow('keypoints on image', image_fast_keypoints)
# Needed so kernel does not crash as well.
cv2.waitKey(0)
# This will just return -1 if not included vs kill window text, possibly for ensuring no background processes?
cv2.destroyAllWindows()